In [5]:
from google import genai
from google.genai.types import Content, HttpOptions, GenerateContentConfig, Tool, Part
from google.genai import Client
import requests
from datetime import date

client = Client(http_options=HttpOptions())

#Define the function
get_exchange_rate_func = {
    "name" : "get_exchange_rate",
    "description"  :"Get latest exchange rate for currencies between countries",
    "parameters" : {
        "type": "object",
        "properties": {
            "date": {
                "type": "string",
                "description": "A date that must always be in YYYY-MM-DD format",
            },
            "from": {
                "type": "string",
                "description": "The currency to convert from ISO 4217 format",
            },
            "to": {
                "type": "string",
                "description": "The currency to convert to in ISO 4217 format",
            }
        },
        "required": ["date","from","to"],
    },
}

exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

generate_content_config = GenerateContentConfig(
    temperature=0.0,
    tools=[exchange_rate_tool],
)

today = date.today()

prompt = (f"""what is the exchange rate from Australia dollar to US dollar for {today}?\
          How much 200 Australian dollar worth in US dollar?""")

function_response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=prompt,
    config=generate_content_config,
)

def get_latest_exchange_rate_from_api():
    try:
        api_url = 'https://api.frankfurter.app/latest'
        currency_api_response = requests.get(api_url, params=function_call.args)
        print(f"Response from API: {currency_api_response.text}\n")
        data =  currency_api_response.json()
        if 'rates' not in data:
            return {"Error": "No rates available"}

        print(f"Response from API: {currency_api_response.text}\n")
        return data

    except requests.HTTPError as e:
        print(e)
        return {"Error!!": f"Error getting exchange rate from API: {e}"}
    except ValueError as e:
        print(e)
        return {"Error: Invalid response from API!!"}



#Validate if the model requested a function call
if function_response.candidates[0].content.parts[0].function_call:
    function_call = function_response.candidates[0].content.parts[0].function_call
    print(f"Function to call : {function_call.name}\n")
    print(f"Function arguments: {function_call.args}\n")

    data_from_api = get_latest_exchange_rate_from_api()

    final_response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[
            prompt,
            Content(role="function", parts=[
                    Part.from_function_response(
                    name=function_call.name,
                    response=data_from_api,
                )
            ]),
        ]
    )
    print(f"Final response : {final_response.text}")
else:
    print("No function calls needed\n")
    print(function_response.text)





Function to call : get_exchange_rate

Function arguments: {'from': 'AUD', 'to': 'USD', 'date': '2026-01-07'}

Response from API: {"amount":1.0,"base":"AUD","date":"2026-01-06","rates":{"USD":0.67197}}

Response from API: {"amount":1.0,"base":"AUD","date":"2026-01-06","rates":{"USD":0.67197}}

Final response : I'm sorry, but I cannot provide exchange rates for a future date like 2026-01-07. Exchange rates are constantly fluctuating and depend on many economic factors, making them impossible to predict accurately in advance.

I can give you the most recent available exchange rate, which is for 2026-01-06.

On 2026-01-06, 1 AUD was worth approximately 0.67197 USD.

Therefore, 200 Australian dollars would be worth:
200 AUD * 0.67197 USD/AUD = **134.394 USD**
